**以下所有的代码块都是这样**
- 当提示 Ready! 后
    - 按 T 开始
    - 按 P 结束

# 1 游戏设置

- 打开游戏 《只狼：影逝二度》
    - 设定
        - 显示和声音
            - **亮度调整**
                - 设置 5
        - 按键设置
            - 移动
                - 移动 前
                    - 设置 W
                - 移动 后
                    - 设置 S
                - 移动 左
                    - 设置 A
                - 移动 右
                    - 设置 D
                - 垫步、（长按）冲刺
                    - 设置 .Shift
                - 跳跃
                    - 设置 Space
            - 视角操作
                - 重置视角/固定目标
                    - 设置 Y
            - 攻击动作
                - **攻击**
                    - 设置 J
                - 使用义手忍具
                    - 设置 .Ctrl
                - **防御**
                    - 设置 K
                - 钩绳
                    - 设置 F
                - 使用道具
                    - 设置 R
        - 图像设定
            - **屏幕模式**
                - 窗口
            - **屏幕分辨率**
                - 1280 x 720
            - 自动绘图调整
                - OFF
            - 质量设定
                - 低

### 校准游戏窗口位置

- 将游戏窗口放在左上方  
![example_01.png](https://github.com/ricagj/pysekiro/blob/main/imgs/example_01.png?raw=true?raw=true)

运行以下代码，然后根据弹出来的窗口调整游戏窗口的位置

In [ ]:
from pysekiro.adjustment import main

main()

还是不懂的话，就看这个gif https://github.com/ricagj/pysekiro/blob/main/imgs/adjustment.gif

![adjustment.png](https://github.com/ricagj/pysekiro/blob/main/imgs/adjustment.png?raw=true)

# 2 开始

## 2.1 无需收集数据直接开始训练

#### 在线学习

~~~python
def learn_online(
    model_weights=None,
    save_path=None,
    reward_curve_save_path='learn_online.png'
    ):
~~~

- **model_weights** 指定模型权重的路径，读取模型权重的参数。默认为None。
- **save_path** 指定模型权重参数保存的路径。参数默认为None，保存为'tmp_weights.h5'。
    - **注：默认是测试模型，若设置该参数，就会开启训练模式**
- **reward_curve_save_path** 奖励曲线保存路径，默认为'learn_online.png'。

In [ ]:
from learn_online import learn_online

In [ ]:
# 打开游戏，校准游戏窗口位置，然后运行这个开始直接在游戏中训练
learn_online(save_path='online_w.h5')

In [ ]:
# 读取模型参数，测试学习的效果（不设置save_path就是测试模式）
learn_online(model_weights='online_w.h5', reward_curve_save_path='learn_online_t.png')

## 2.2 需要收集数据

#### 2.2.1 收集数据

- 数据集
    - 格式 n行2列 每行由[**图像**, **按键**]组成
        - 图像
            - 区域：左上角处 1280x720
            - 存储标准：480x270, BGR
        - 按键
            - "攻击"、"防御"、垫步"  、"跳跃" 、"其他" 、"使用道具"、"移动 前"、"移动 后"、"移动 左"、"移动 右"
            - J    \ K     \ LSHIFT \ SPACE \ 无操作 \ R        \ W       \ S        \ A       \ D
            - 存储格式：列表
    - 每个数据集都是一次完整的战斗
        - 要求 禁所有忍具，禁所有流派招式，禁复活（出现这些行为的数据集要及时删除，不然会影响训练效果）
        - 建议 双难（交护符和钟鬼） 
        - 可选 使用伤药葫芦等生命回复道具或夜叉戮之降临等BUFF道具

1. 建议使用《只狼：影逝二度》v1.06版本
2. 建议通过鬼佛处“再战稀世强者”收集数据
3. 建议在战斗开始时开始搜集数据，战斗结束时停止，保证每一个数据集都是一场完整的战斗。

- 以下是收集“苇名弦一郎”战斗数据的示例。
- 如果想收集其他Boss的战斗数据，那就把target改成对应的Boss的名字

In [ ]:
from pysekiro.collect_data import Data_collection

# target = 'Genichiro_Ashina' # 苇名弦一郎
target = 'Inner_Genichiro' # 心中的弦一郎
# target = 'Isshin,_the_Sword_Saint' # 剑圣一心
# target = 'Inner_Isshin' # 心中的一心

In [ ]:
c = Data_collection(target)
c.collect_data()

#### 2.2.2 开始训练

- 以下是用“苇名弦一郎”战斗数据学习的示例。
- 如果想用其他Boss的战斗数据学习，那就把target改成对应的Boss的名字

In [ ]:
from learn_offline import learn_offline
from learn_online import learn_online

target = 'Genichiro_Ashina' # 苇名弦一郎
# target = 'Inner_Genichiro' # 心中的弦一郎
# target = 'Isshin,_the_Sword_Saint' # 剑圣一心
# target = 'Inner_Isshin' # 心中的一心

##### 2.2.2.1 离线学习

~~~python
def learn_offline(
    target,
    start=1,
    end=1,
    model_weights=None,
    save_path=None,
    reward_curve_save_path='learn_offline.png'
    ):
~~~

- **target** 指定训练的对象
- **start** 指定第一个训练的数据集的序号
- **end** 指定最后一个训练的数据集的序号
    - 数据集的序号指的是数据集文件名里唯一的数字。e.g. training_data-19.npy 序号为 19
    - 注：必须 start < end
- **model_weights** 指定模型权重的路径，读取模型权重的参数。默认为None。也可用以支持增量学习。
    - 当你搜集新的数据集之后想继续训练，设置该参数以继续训练。
    - 当你在训练过程中发生了某种意外终止了训练，设置该参数并把发生意外时正在训练的那个数据集的序号设置为start以继续训练。
- **save_path** 指定模型权重参数保存的路径。参数默认为None，保存为'tmp_weights.h5'。
- **reward_curve_save_path** 奖励曲线保存路径，默认为'learn_offline.png'。

In [ ]:
# 指定 start 和 end 后开始训练
learn_offline(target, start=1, end=1, save_path='offline_w.h5')

In [ ]:
# 读取离线学习的模型参数，测试学习的效果（不设置save_path就是测试模式）
learn_online(model_weights='offline_w.h5', reward_curve_save_path='learn_offline_t.png')

##### 2.2.2.2 在线学习

跟前面那个在线学习的区别是读取了离线学习的模型参数  
这意味着前面那个在线学习是零基础开始而这里的在线学习具备了一定的经验

In [ ]:
# 读取离线学习的模型参数，然后在游戏中继续训练
learn_online(model_weights='offline_w.h5', save_path='on_off_w.h5')

In [ ]:
# 读取模型参数，测试学习的效果（不设置save_path就是测试模式）
learn_online(model_weights='on_off_w.h5', reward_curve_save_path='on_off_t.png')